In [19]:
from inspect import Signature, Parameter


def auto_label(self):
    cname = self.__class__.__qualname__
    signature = Signature.from_callable(self.__init__)
    args, keyword_only = [], False

    for p in signature.parameters.values():
        v = getattr(self, p.name, p.default)

        if p.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise ValueError(f"Unsupported parameter type {p.kind}")

        if p.kind == Parameter.KEYWORD_ONLY:
            keyword_only = True
        elif isinstance(p.default, (type(None), str, bool)):
            keyword_only = True

        if v == p.default:
            # skip argument if not equal to default
            if keyword_only or not isinstance(v, (int, float)):
                keyword_only = True
                continue

        if keyword_only:
            args.append(f"{p.name}={v!r}")
        else:
            args.append(f"{v!r}")

    args = ", ".join(args)

    return f"{cname}({args})"


class Indicator:
    """Implements a basic __repr__ based on __init__ signature"""

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        cls.__repr__ = auto_label




In [20]:
from dataclasses import dataclass, field

@dataclass(frozen=True)
class EMA(Indicator):
    period: int = 20
    adjust : bool = False
    item: str = field(default=None)

    def __call__(self, prices):
        print(self.__class__.__name__, self.period, self.item)
        return prices


for i in EMA(20), EMA(adjust=True), EMA(20, item="close"):
    print(i)



EMA(20)
EMA(20, adjust=True)
EMA(20, item='close')


In [1]:
def custom_label(obj, **params) -> str:
    """ custom object label according to params
        param = None for required/positional params
        param = <default> for optional/keyword params
    """
    cname = obj.__class__.__name__
    args, kw_only = [], False
    for k, v in params.items():
        value = getattr(obj, k, v)
        if value is None or v is not None:
            kw_only = True
        if not kw_only:
            args.append(f"{value!r}")
        elif value != v:
            args.append(f"{k}={value!r}")
    args = ", ".join(args)
    return f"{cname}({args})"




In [2]:
from dataclasses import dataclass, field


@dataclass(frozen=True)
class EMA:
    period: int = 20
    item: str = field(default=None)

    def __str__(self) -> str:
        return custom_label(self, period=None)

ema = EMA(20, item="close")

print(ema)



EMA(20)
